# ETL Silver → Gold

Pipeline ETL para popular o Star Schema da camada Gold a partir dos dados limpos do Silver Layer.

**Objetivo:** Transformar dados normalizados em estrutura dimensional otimizada para análises de BI.

## 1. Imports

Bibliotecas necessárias para o ETL.

In [1]:
import pandas as pd
import psycopg2
from psycopg2.extras import execute_batch
from datetime import datetime

## 2. Configuração

Parâmetros de conexão ao banco de dados PostgreSQL.

In [2]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'airline_delays',
    'user': 'postgres',
    'password': 'postgres'
}

## 3. Conectar e Criar Schema DW

Abre conexão com PostgreSQL e executa o DDL para criar as tabelas dimensionais e fato.

In [3]:
print("Conectando ao PostgreSQL...")
conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

print("Executando DDL da camada DW...")
with open('../Data Layer/gold/ddl.sql', 'r', encoding='utf-8') as f:
    ddl_sql = f.read()
    cur.execute(ddl_sql)
    conn.commit()

print("Schema DW criado com sucesso")
cur.close()

Conectando ao PostgreSQL...
Executando DDL da camada DW...
Schema DW criado com sucesso


## 4. Carregar Dados do Silver

Leitura dos dados da tabela `silver.airline_delays`.

In [4]:
print("Carregando dados do Silver...")

query = """
    SELECT 
        year,
        month,
        carrier,
        carrier_name,
        airport,
        airport_name,
        arr_flights,
        arr_del15,
        carrier_ct,
        weather_ct,
        nas_ct,
        security_ct,
        late_aircraft_ct,
        arr_cancelled,
        arr_diverted,
        arr_delay,
        carrier_delay,
        weather_delay,
        nas_delay,
        security_delay,
        late_aircraft_delay
    FROM silver.airline_delays
    ORDER BY year, month, carrier, airport
"""

df = pd.read_sql_query(query, conn)
print(f"Carregados {len(df):,} registros do Silver")
print(f"\nInfo do DataFrame:")
print(df.info())

Carregando dados do Silver...


/tmp/ipykernel_13756/669887564.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Carregados 171,666 registros do Silver

Info do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171666 entries, 0 to 171665
Data columns (total 21 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   year                 171666 non-null  int64  
 1   month                171666 non-null  int64  
 2   carrier              171666 non-null  object 
 3   carrier_name         171666 non-null  object 
 4   airport              171666 non-null  object 
 5   airport_name         171666 non-null  object 
 6   arr_flights          171426 non-null  float64
 7   arr_del15            171223 non-null  float64
 8   carrier_ct           171426 non-null  float64
 9   weather_ct           171426 non-null  float64
 10  nas_ct               171426 non-null  float64
 11  security_ct          171426 non-null  float64
 12  late_aircraft_ct     171426 non-null  float64
 13  arr_cancelled        171426 non-null  float64
 14  arr_diver

## 5. Preparar Cursor

Cria cursor para operações no banco.

In [5]:
cur = conn.cursor()
print("Cursor criado")

Cursor criado


## 6. Limpar Tabelas DW

Remove dados antigos para reprocessamento.

In [6]:
print("Limpando tabelas DW...")
cur.execute("TRUNCATE TABLE dw.FCT_FIT_DLS, dw.DIM_CRE, dw.DIM_ARR, dw.DIM_TME CASCADE;")
conn.commit()

Limpando tabelas DW...


## 7. Dimensão Tempo (DIM_TME)

Popula a dimensão temporal com atributos calculados.

In [ ]:
print("Populando DIM_TME...")

time_data = df[['year', 'month']].drop_duplicates().sort_values(['year', 'month'])

data = []
meses_nomes = [
    'Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho',
    'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro'
]

for _, row in time_data.iterrows():
    year = int(row['year'])
    month = int(row['month'])
    
    trimestre = (month - 1) // 3 + 1
    semestre = 1 if month <= 6 else 2
    mes_nome = meses_nomes[month - 1]
    mes_ano = f"{year}-{month:02d}"
    ano_trimestre = f"{year}-Q{trimestre}"
    
    data.append((
        year,
        month,
        trimestre,
        semestre,
        mes_nome,
        mes_ano,
        ano_trimestre
    ))

execute_batch(cur, """
    INSERT INTO dw.DIM_TME 
    (YAR, MNH, TIS, SMT, MES_NME, MES_ANO, ANO_TIS)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (YAR, MNH) DO NOTHING
""", data)
conn.commit()

print(f"DIM_TME populada com {len(data):,} registros")

Populando dim_time...
DIM_TME populada com 121 registros


## 8. Dimensão Companhia Aérea (DIM_CRE)

Popula a dimensão de companhias aéreas.

In [8]:
print("Populando DIM_CRE...")

carriers = df[['carrier', 'carrier_name']].drop_duplicates()

data = []
for _, row in carriers.iterrows():
    carrier_code = str(row['carrier']) if pd.notna(row['carrier']) else 'UNKNOWN'
    carrier_name = str(row['carrier_name']) if pd.notna(row['carrier_name']) else 'Unknown Carrier'
    data.append((carrier_code, carrier_name))

execute_batch(cur, """
    INSERT INTO dw.DIM_CRE (CRE_CDE, CRE_NME)
    VALUES (%s, %s)
    ON CONFLICT (CRE_CDE) DO UPDATE SET
        CRE_NME = EXCLUDED.CRE_NME
""", data)
conn.commit()

print(f"DIM_CRE populada com {len(data):,} registros")

Populando DIM_CRE...
DIM_CRE populada com 23 registros


## 9. Dimensão Aeroporto (DIM_ARR)

Popula a dimensão de aeroportos.

In [9]:
print("Populando DIM_ARR...")

airports = df[['airport', 'airport_name']].drop_duplicates()

data = []
for _, row in airports.iterrows():
    airport_code = str(row['airport']) if pd.notna(row['airport']) else 'UNKNOWN'
    airport_name = str(row['airport_name']) if pd.notna(row['airport_name']) else 'Unknown Airport'
    data.append((airport_code, airport_name))

execute_batch(cur, """
    INSERT INTO dw.DIM_ARR (ARR_CDE, ARR_NME)
    VALUES (%s, %s)
    ON CONFLICT (ARR_CDE) DO UPDATE SET
        ARR_NME = EXCLUDED.ARR_NME
""", data)
conn.commit()

print(f"DIM_ARR populada com {len(data):,} registros")

Populando DIM_ARR...
DIM_ARR populada com 419 registros


## 10. Buscar Chaves Surrogate (SRK)

Cria dicionários para mapear códigos naturais → surrogate keys das 3 dimensões.

In [10]:
print("Buscando surrogate keys...")

cur.execute("SELECT CRE_CDE, SRK_CRE FROM dw.DIM_CRE")
carrier_to_key = dict(cur.fetchall())
print(f"  {len(carrier_to_key)} carriers mapeados")

cur.execute("SELECT ARR_CDE, SRK_ARR FROM dw.DIM_ARR")
airport_to_key = dict(cur.fetchall())
print(f"  {len(airport_to_key)} airports mapeados")

cur.execute("SELECT YAR, MNH, SRK_TME FROM dw.DIM_TME")
time_to_key = {(year, month): key for year, month, key in cur.fetchall()}
print(f"  {len(time_to_key)} períodos mapeados")

Buscando surrogate keys...
  21 carriers mapeados
  395 airports mapeados
  121 períodos mapeados


## 11. Tabela Fato (FCT_FIT_DLS)

Popula a tabela fato com **1 registro por combinação** carrier-airport-time, mantendo todas as causas de atraso como colunas.

In [11]:
print("Populando FCT_FIT_DLS...")

data = []
skipped = 0

for _, row in df.iterrows():
    carrier_srk = carrier_to_key.get(str(row['carrier']))
    airport_srk = airport_to_key.get(str(row['airport']))
    time_srk = time_to_key.get((int(row['year']), int(row['month'])))
    
    if not (carrier_srk and airport_srk and time_srk):
        skipped += 1
        continue
    
    # Converter valores para float, tratando NaN
    arr_flights = float(row['arr_flights']) if pd.notna(row['arr_flights']) else 0
    arr_del15 = float(row['arr_del15']) if pd.notna(row['arr_del15']) else 0
    arr_cancelled = float(row['arr_cancelled']) if pd.notna(row['arr_cancelled']) else 0
    arr_diverted = float(row['arr_diverted']) if pd.notna(row['arr_diverted']) else 0
    arr_delay = float(row['arr_delay']) if pd.notna(row['arr_delay']) else 0
    
    carrier_ct = float(row['carrier_ct']) if pd.notna(row['carrier_ct']) else 0
    weather_ct = float(row['weather_ct']) if pd.notna(row['weather_ct']) else 0
    nas_ct = float(row['nas_ct']) if pd.notna(row['nas_ct']) else 0
    security_ct = float(row['security_ct']) if pd.notna(row['security_ct']) else 0
    late_aircraft_ct = float(row['late_aircraft_ct']) if pd.notna(row['late_aircraft_ct']) else 0
    
    carrier_delay = float(row['carrier_delay']) if pd.notna(row['carrier_delay']) else 0
    weather_delay = float(row['weather_delay']) if pd.notna(row['weather_delay']) else 0
    nas_delay = float(row['nas_delay']) if pd.notna(row['nas_delay']) else 0
    security_delay = float(row['security_delay']) if pd.notna(row['security_delay']) else 0
    late_aircraft_delay = float(row['late_aircraft_delay']) if pd.notna(row['late_aircraft_delay']) else 0
    
    # Calcular métricas
    delay_rate = (arr_del15 / arr_flights * 100) if arr_flights > 0 else 0
    cancellation_rate = (arr_cancelled / arr_flights * 100) if arr_flights > 0 else 0
    diversion_rate = (arr_diverted / arr_flights * 100) if arr_flights > 0 else 0
    avg_delay_minutes = (arr_delay / arr_flights) if arr_flights > 0 else 0
    on_time_rate = 100 - delay_rate
    
    data.append((
        carrier_srk,
        airport_srk,
        time_srk,
        arr_flights,
        arr_del15,
        arr_cancelled,
        arr_diverted,
        arr_delay,
        carrier_ct,
        weather_ct,
        nas_ct,
        security_ct,
        late_aircraft_ct,
        carrier_delay,
        weather_delay,
        nas_delay,
        security_delay,
        late_aircraft_delay,
        round(delay_rate, 2),
        round(cancellation_rate, 2),
        round(diversion_rate, 2),
        round(avg_delay_minutes, 2),
        round(on_time_rate, 2)
    ))

print(f"  Processando {len(data):,} registros...")

execute_batch(cur, """
    INSERT INTO dw.FCT_FIT_DLS 
    (SRK_CRE, SRK_ARR, SRK_TME,
     ARR_FIT, ARR_DEL, ARR_CNL, ARR_DVR, ARR_DLY,
     CRE_CT, WAE_CT, NAS_CT, SCR_CT, LTE_ARA_CT,
     CRE_DLY, WAE_DLY, NAS_DLY, SCR_DLY, LTE_ARA_DLY,
     DLY_RTE, CNL_RTE, DVS_RTE, AVG_DLY_MNE, ONT_RTE)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", data, page_size=1000)
conn.commit()

print(f"FCT_FIT_DLS populada com {len(data):,} registros")
if skipped > 0:
    print(f"AVISO: {skipped} registros ignorados (FKs inválidas)")

Populando FCT_FIT_DLS...
  Processando 171,666 registros...
FCT_FIT_DLS populada com 171,666 registros


## 12. Finalizar Conexão

Fecha cursor e conexão com o banco de dados.

In [12]:
cur.close()
conn.close()
print("Conexão encerrada.")

Conexão encerrada.
